In [ ]:
#Title: Estimates of Emergency Department Visits in the United States from 2016-2022

#==used variables==

#df_all_primary_rates: All diagnoses, primary, visit rates
#df_all_primary_count: All diagnoses, primary, visit count
#df_all_reasons_rates: All reasons, reason, visit rates
#df_all_reasons_count: All reasons, reason, visit counts
#df_detailed_primary_rates: Detailed, primary, visit rates
#df_detailed_primary_count:Detailed, primary, counts
#df_detailed_reasons_rates:Detailed, reason, rates
#df_detailed_reasons_count: Detailed, reason, counts

#==importing libraries==

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


"""
**Data Loading and Initial Cleaning**
* **Purpose:** Load the raw dataset and perform basic cleaning steps to prepare it for analysis.
* **Steps Taken:**
  * Load the CSV file.
  * View data using `.head()`, `.describe()`, `.info()`.
  * Drop rows where estimates are **not reliable** (`Reliable == "No"`).
  * Check and remove duplicates.
"""


df= pd.read_csv('Estimates_of_Emergency_Department_Visits_in_the_United_States_from_2016-2022.csv')
df.head()
df.describe()
df.info()

#checking for nulls> Nulss exists where "reliable"== No
df.isnull().sum()

## droping all rows where reliable== no
df= df.drop(df[df['Reliable']=='No'].index)

##checking if there other nulls
df.isnull().sum()

##checking for duplicates
df.duplicated().sum()
#----------------------------
#saving cleaned data
df.to_csv('CLEANED_Estimates_of_Emergency_Department_Visits_in_the_United_States_from_2016-2022.csv', index=False)

#------------------

### ==Data Splitting==
"""
Purpose: Split the cleaned data into multiple smaller DataFrames based on:
Diagnosis vs Reason for Visit
Primary vs Detailed
Visit Rate vs Visit Count
Explanation:
Uses boolean masks to filter for specific combinations like:
“All Primary diagnoses rates”
“ALL reason for visit counts”
Result: 8 datasets (4 for visit counts, 4 for visit rates).
"""


mask_visit_count= df['Estimate Type']== 'Visit count'
mask_visit_rates= df['Estimate Type']== 'Visit rate (per 1,000 people)'

# #Separate data into all diagnosis-visit rate, all-diagnosis- visit cout, detailed- visit count, detailed- visit rates
# #df_all_diagnosis -- it contains data that got total' rows aka summary rows

mask_all_diagnosis_primary= df['Measure'] == 'All diagnoses'
mask_all_reasons= (df['Measure Type'] == 'By reason for visit') & (df['Measure'] == 'All reasons')

##df_detailed -- it contains data with groups and subgroups
## Detailed diagnoses only (excluding 'All diagnoses'), from Measure Type = "By primary diagnosis"

mask_detailed_primary= (df['Measure Type'] == 'By primary diagnosis') & (df['Measure'] != 'All diagnoses')
mask_detailed_reasons= (df['Measure Type'] == 'By reason for visit') & (df['Measure'] != 'All reasons')

# #seperating into 4 tables> all diagnosis, all reasons, rates/counts

df_all_primary_rates= df[mask_all_diagnosis_primary & mask_visit_rates].copy()
df_all_primary_count= df[mask_all_diagnosis_primary & mask_visit_count].copy()
df_all_reasons_rates=df[mask_all_reasons & mask_visit_rates].copy()
df_all_reasons_count=df[mask_all_reasons & mask_visit_count].copy()
# #seperating into 4 tables> detailed diagnosis, reasons, rates/counts
df_detailed_primary_rates=df[mask_detailed_primary & mask_visit_rates].copy()
df_detailed_primary_count= df[mask_detailed_primary & mask_visit_count].copy()
df_detailed_reasons_rates= df[mask_detailed_reasons & mask_visit_rates].copy()
df_detailed_reasons_count=df[mask_detailed_reasons & mask_visit_count].copy()
#------------------------
"""
Normalization for Visit Rates
Purpose: Convert visit rate values from per 1,000 people to per person.
Why: Enables direct comparison and statistical calculations using standard units.
"""


columns_to_normalize = ['Estimate', 'Standard Error', 'Lower 95% CI', 'Upper 95% CI']
df_all_primary_rates[columns_to_normalize] = df_all_primary_rates[columns_to_normalize] / 1000
df_detailed_primary_rates[columns_to_normalize] = df_detailed_primary_rates[columns_to_normalize] / 1000
df_detailed_reasons_rates[columns_to_normalize] = df_detailed_reasons_rates[columns_to_normalize] / 1000
df_all_reasons_rates[columns_to_normalize] = df_all_reasons_rates[columns_to_normalize] / 1000
#df.to_csv('Estimates_of_Emergency_Department_Visits_in_the_United_States_from_2016-2022Cleaned.csv', index=False)

#-----------------------

#==Analysis===
"""
Green Flag Condition #1: Tight Confidence Interval (CI)
Purpose: Flag estimates with tight CI ranges — an indicator of precision.
Method: Calculate CI Width and CI Width as a % of Estimate.
Flag as "Yes" if CI% is under a threshold (e.g., 15%).
"""


tight_ci_threshold= 15
def tight_ci_filter(df, threshold= 15):
df= df.copy()
df['CI WIDTH']= df['Upper 95% CI']- df['Lower 95% CI']
df['CI WIDTH PERCENTAGE']= (df['CI WIDTH']/ df['Estimate'] )* 100
df['CI TIGHT?']= np.where(df['CI WIDTH PERCENTAGE']< threshold, 'Yes', 'No')
return df

##applying tight_ci tag on all tables

df= tight_ci_filter(df)

#------------------------------------------
"""
Green Flag Condition #2: Statistically Significant Changes- Two-sample Z-test
Purpose: Identify whether the change in estimates between two years is statistically significant.
Calculates:
Difference of estimates
Standard Error of the difference
95% Confidence Interval
Flags significance if the CI does not include 0.
"""


##==comapre with CI code==
def compare_with_ci(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    on_cols,             # List or single column name(s) to join on, e.g. ['Measure', 'Group']
    estimate_col: str = 'Estimate',
    se_col: str = 'Standard Error',
    z: float = 1.96,
    suffixes=('_1', '_2')
) -> pd.DataFrame:
    """
    Compare two DataFrames of estimates with standard errors,
    calculating the difference, 95% CI for difference, and significance.

    Parameters:
    - df1, df2: DataFrames to compare (e.g. year 2016 and 2022)
    - on_cols: Column(s) to join on (e.g. ['Measure', 'Group'])
    - estimate_col: name of estimate column
    - se_col: name of standard error column
    - z: z-score for confidence interval (1.96 for 95%)
    - suffixes: suffixes for merged columns to distinguish dfs

    Returns:
    - merged DataFrame with columns:
      on_cols + estimate_col + se_col from each df,
      Difference, CI Lower, CI Upper, Significant (bool)
    """

    # Make sure on_cols is a list
    if isinstance(on_cols, str):
        on_cols = [on_cols]

    # Merge dataframes on the specified keys
    merged = pd.merge(
        df1[on_cols + [estimate_col, se_col]],
        df2[on_cols + [estimate_col, se_col]],
        on=on_cols,
        suffixes=suffixes
    )

    est1 = estimate_col + suffixes[0]
    est2 = estimate_col + suffixes[1]
    se1 = se_col + suffixes[0]
    se2 = se_col + suffixes[1]

    # Calculate difference of estimates
    merged['Difference'] = merged[est1] - merged[est2]

    # Calculate standard error of the difference
    merged['SE_diff'] = np.sqrt(merged[se1]**2 + merged[se2]**2)

    # Margin of error
    merged['Margin'] = z * merged['SE_diff']

    # Confidence Interval bounds
    merged['CI Lower'] = merged['Difference'] - merged['Margin']
    merged['CI Upper'] = merged['Difference'] + merged['Margin']

    # Significance: True if CI does not include zero
    merged['Significant'] = ~((merged['CI Lower'] <= 0) & (merged['CI Upper'] >= 0))

    return merged

#------------------------
"""
Function: condition_specific_significant_subgroup_trends
Purpose: Detect statistically significant trends for each condition across groups and subgroups between two years.
How:
For each subgroup within a condition:
Compares early vs late year.
Uses compare_with_ci to determine significance.
Records result and calculates % change.
"""


def condition_specific_significant_subgroup_trends(
    df,
    year_earlier,
    year_later,
    estimate_col="Estimate",
    se_col="Standard Error"
):
    all_conditions = df['Measure'].unique()

    results = []

    for condition in all_conditions:
        cond_df = df[df['Measure'] == condition]
        for group in cond_df['Group'].unique():
            for subgroup in cond_df[cond_df['Group'] == group]['Subgroup'].unique():
                df_early = cond_df[
                    (cond_df['Year'] == year_earlier) &
                    (cond_df['Group'] == group) &
                    (cond_df['Subgroup'] == subgroup)
                ]
                df_late = cond_df[
                    (cond_df['Year'] == year_later) &
                    (cond_df['Group'] == group) &
                    (cond_df['Subgroup'] == subgroup)
                ]

                if df_early.empty or df_late.empty:
                    continue

                result = compare_with_ci(
                    df1=df_late,
                    df2=df_early,
                    on_cols=['Measure', 'Measure Type', 'Group', 'Subgroup'],
                    estimate_col=estimate_col,
                    se_col=se_col
                )

                if result.iloc[0]['Significant']:
                    est1 = df_early[estimate_col].values[0]
                    est2 = df_late[estimate_col].values[0]
                    pct_change = ((est2 - est1) / est1) * 100
                    direction_arrow = "↑" if pct_change > 0 else "↓"
                    direction_label = "Increase" if pct_change > 0 else "Decrease"

                    results.append({
                        "Year Range": f"{year_earlier}–{year_later}",
                        "Condition": condition,
                        "Measure Type": df_early["Measure Type"].values[0],
                        "Estimate Type": df_early["Estimate Type"].values[0],
                        "Group": group,
                        "Subgroup": subgroup,
                        "% Change": f"{direction_arrow} {abs(pct_change):.2f}%",
                        "Direction": direction_label,
                        "Estimate(Early)": f"{est1:,.0f}",
                        "Estimate(Late) ": f"{est2:,.0f}",
                        "CI Lower (Early)": df_early["Lower 95% CI"].values[0],
                        "CI Upper (Early)": df_early["Upper 95% CI"].values[0],
                        "CI Lower (Late)": df_late["Lower 95% CI"].values[0],
                        "CI Upper (Late)": df_late["Upper 95% CI"].values[0]
                    })

    return pd.DataFrame(results)

results = condition_specific_significant_subgroup_trends(
    df=df_all_reasons_rates,
    year_earlier=2016,
    year_later=2022
)

display(results)
#----------
## ==Visualizing==
"""
Visual 1: ED Visit Counts Compared to 2022
Purpose: Plot total ED visit counts per year and flag years that are significantly different from 2022.
How: Calculates CI bounds for each year.
Uses compare_with_ci to flag significance.
Plots using matplotlib:
Lines for trends
Colored dots for significance (orange = significant)
"""


def run_year_vs_2022_significance_flagger_with_plot_cis(df, group='Total', subgroup='All visits', end_year=2022):
    """
    Compares each earlier year directly against the final year (default: 2022)
    for a given group & subgroup. Flags significance and prepares CI for plotting.

    Returns a dataframe with:
      - CI_Lower_Plot
      - CI_Upper_Plot
      - Significant_Vs2022 (bool)
    """
    # Filter to only relevant rows
    df_filtered = df[
        (df['Group'] == group) &
        (df['Subgroup'] == subgroup)
    ].copy()

    df_filtered = df_filtered.sort_values('Year').reset_index(drop=True)

    # Add CI bounds for plotting
    df_filtered['CI_Lower_Plot'] = df_filtered['Estimate'] - 1.96 * df_filtered['Standard Error']
    df_filtered['CI_Upper_Plot'] = df_filtered['Estimate'] + 1.96 * df_filtered['Standard Error']

    # Initialize flag column
    df_filtered['Significant_Vs2022'] = False

    # Get 2022 data
    df_2022 = df_filtered[df_filtered['Year'] == end_year]

    # Loop through all years except 2022
    for year in df_filtered['Year'].unique():
        if year == end_year:
            continue

        df_year = df_filtered[df_filtered['Year'] == year]

        # Compare this year to 2022
        result = compare_with_ci(
            df1=df_2022,
            df2=df_year,
            on_cols=['Measure', 'Measure Type', 'Group', 'Subgroup'],
            estimate_col='Estimate',
            se_col='Standard Error',
            z=1.96
        )

        # Extract flags
        significant_mask = result.set_index(['Measure', 'Measure Type']).loc[:, 'Significant']

        # Mark significance for this year
        indices_to_flag = df_filtered[
            (df_filtered['Year'] == year) &
            (df_filtered.set_index(['Measure', 'Measure Type']).index.isin(significant_mask[significant_mask].index))
        ].index

        df_filtered.loc[indices_to_flag, 'Significant_Vs2022'] = True

    return df_filtered

df_flagged = run_year_vs_2022_significance_flagger_with_plot_cis(
    df=df_all_reasons_count,
    group='Total',
    subgroup='All visits',
    end_year=2022
)

#-----
import matplotlib.pyplot as plt

def plot_ed_visits_vs_2022(df_flagged, title="ED Visit Counts Compared to 2022"):
    years = df_flagged['Year']
    estimates = df_flagged['Estimate']
    lower = df_flagged['CI_Lower_Plot']
    upper = df_flagged['CI_Upper_Plot']
    errors = upper - estimates

    colors = df_flagged['Significant_Vs2022'].map({True: '#FF7A30', False: 'black'})
    alphas = df_flagged['Significant_Vs2022'].map({True: 1.0, False: 0.8})

    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot only the line (no marker)
    ax.errorbar(
        years, estimates,
        yerr=errors,
        fmt='-',            # no marker
        color='black',
        ecolor='#E9E3DF',
        elinewidth=2,
        capsize=4,
        zorder=1  # draw line below
    )

    # Overlay custom-colored points
    for i in range(len(df_flagged)):
        ax.plot(
            years.iloc[i], estimates.iloc[i],
            'o',
            color=colors.iloc[i],
            alpha=alphas.iloc[i],
            markersize=8,
            zorder=2  # draw on top
        )

    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Year")
    ax.set_ylabel("Visit Count")
    ax.grid(True, linestyle='--', alpha=0.5)

    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Significant vs 2022', markerfacecolor='#FF7A30', markersize=8),
        Line2D([0], [0], marker='o', color='w', label='Not significant', markerfacecolor='#000000', alpha=0.8, markersize=8)
    ]
    ax.legend(handles=legend_elements, title="Change vs 2022")

    plt.tight_layout()
    plt.show()


plot_ed_visits_vs_2022(df_flagged, title="Overall ED Visit Counts Compared to 2022(Reasons)")

print(df_flagged[['Year', 'Estimate', 'CI_Lower_Plot', 'CI_Upper_Plot', 'Significant_Vs2022']])

#---------------------------------------------------------------------------
"""
Visual 2: Demographic Change Comparison – Counts

Purpose: Visualize % change in visit counts for key subgroups across selected periods.

How:

Uses grouped bar plots split by:

Primary Diagnosis

Reason for Visit

X-axis shows subgroups + period

Y-axis shows % increase
"""


# Data added with years order
data = [
    # Primary Diagnosis (exclude 'Total')
    {'Category': 'Primary Diagnosis', 'Subgroup': 'Male', 'Period': '2018-2022', 'PercentChange': 22.40},
    {'Category': 'Primary Diagnosis', 'Subgroup': '0-17 years old', 'Period': '2020-2022', 'PercentChange': 38.58},
    {'Category': 'Primary Diagnosis', 'Subgroup': '65+ years old', 'Period': '2016-2022', 'PercentChange': 42.53},
    {'Category': 'Primary Diagnosis', 'Subgroup': '65+ years old', 'Period': '2017-2022', 'PercentChange': 47.17},
    {'Category': 'Primary Diagnosis', 'Subgroup': '65+ years old', 'Period': '2018-2022', 'PercentChange': 41.98},
    {'Category': 'Primary Diagnosis', 'Subgroup': 'Non-Hispanic White', 'Period': '2018-2022', 'PercentChange': 24.79},
    {'Category': 'Primary Diagnosis', 'Subgroup': 'Medicare', 'Period': '2016-2022', 'PercentChange': 32.05},
    # Reasons for Visit (added missing rows)
    {'Category': 'Reasons for Visit', 'Subgroup': 'Male', 'Period': '2018-2022', 'PercentChange': 22.40},
    {'Category': 'Reasons for Visit', 'Subgroup': '0-17 years old', 'Period': '2020-2022', 'PercentChange': 38.58},  # added missing
    {'Category': 'Reasons for Visit', 'Subgroup': '65+ years old', 'Period': '2016-2022', 'PercentChange': 42.53},  # added missing
    {'Category': 'Reasons for Visit', 'Subgroup': '65+ years old', 'Period': '2017-2022', 'PercentChange': 47.17},
    {'Category': 'Reasons for Visit', 'Subgroup': '65+ years old', 'Period': '2018-2022', 'PercentChange': 41.98},
    {'Category': 'Reasons for Visit', 'Subgroup': 'Non-Hispanic White', 'Period': '2018-2022', 'PercentChange': 24.79},
    {'Category': 'Reasons for Visit', 'Subgroup': 'Medicare', 'Period': '2016-2022', 'PercentChange': 32.05},
    {'Category': 'Reasons for Visit', 'Subgroup': 'Medicare', 'Period': '2017-2022', 'PercentChange': 33.35},
    {'Category': 'Reasons for Visit', 'Subgroup': 'Medicare', 'Period': '2018-2022', 'PercentChange': 36.58},
    {'Category': 'Reasons for Visit', 'Subgroup': 'Other-payment source', 'Period': '2018-2022', 'PercentChange': 49.73},
]

df_plot = pd.DataFrame(data)

# Ensure the period ordering
period_order = ['2016-2022', '2017-2022', '2018-2022', '2020-2022']
df_plot['Period'] = pd.Categorical(df_plot['Period'], categories=period_order, ordered=True)


# Prepare plotting grouped bar chart with matplotlib for precise control
categories = df_plot['Category'].unique()
subgroups = sorted(df_plot['Subgroup'].unique())
periods = period_order

# Width and positioning
bar_width = 0.18
n_periods = len(periods)

fig, axs = plt.subplots(1, len(categories), figsize=(16, 7), sharey=True)
if len(categories) == 1:
    axs = [axs]

for ax, cat in zip(axs, categories):
    df_cat = df_plot[df_plot['Category'] == cat]
    subgroups = df_cat[df_cat['PercentChange'].notna()]['Subgroup'].unique().tolist()
    x = np.arange(len(subgroups))  # x-axis positions for bars


    # Plot each period offset by bar_width
    for i, period in enumerate(periods):
        vals = []
        for sg in subgroups:
            val = df_cat[(df_cat['Subgroup'] == sg) & (df_cat['Period'] == period)]
            if not val.empty:
                vals.append(val['PercentChange'].values[0])
            else:
                vals.append(0)  # or np.nan if you want gaps

        ax.bar(x + i*bar_width, vals, width=bar_width, label=period)

    ax.set_xticks(x + bar_width*(n_periods-1)/2)
    ax.set_xticklabels(subgroups, rotation=45, ha='right')

    ax.set_title(cat)
    ax.set_ylabel('Percent Change (%)')
    ax.axhline(0, color='black', linewidth=0.8)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

# Legend with specified order
handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles, labels, title='Period', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.suptitle('Significant Increases in ED Visits Counts by Subgroup', fontsize=16, weight='bold')
plt.tight_layout(rect=[0, 0, 0.85, 0.95])
plt.savefig("EDsubgroupALLCOUNTS.png", dpi=300, bbox_inches='tight')
plt.show()

#----------------------
"""
Visual 3: Demographic Change Comparison – Rates
Purpose: Show % change in visit rates (not counts) for various subgroups.
How:
Bar chart comparison:
Left bar: All Diagnosis rates
Right bar: All Reasons rates
X-axis: Subgroup (with time range)
Y-axis: Percent increase

"""


groups = ['Male', '0-17 years old', '65+ years old', '65+ years old', 'Non-Hispanic white', 'Other-payment source']
years = ['2018-2022', '2020-2022', '2017-2022', '2018-2022', '2018-2022', '2018-2022']
percent_changes = [18.97, 39.22, 29.28, 28.71, 25.53, 80.51]

# For "ALL REASONS" category:
groups_reasons = ['Male', '0-17 years old', '65+ years old', '65+ years old', 'Non-Hispanic white', 'Other-payment source']
years_reasons = ['2018-2022', '2020-2022', '2017-2020', '2018-2020', '2018-2022', '2018-2022']
percent_changes_reasons = [18.97, 39.22, 29.28, 28.71, 25.53, 80.51]

x = np.arange(len(groups))  # label locations
width = 0.35  # bar width

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width/2, percent_changes, width, label='All Diagnosis - Visit Rates', color='#FF7A30')
bars2 = ax.bar(x + width/2, percent_changes_reasons, width, label='All Reasons - Visit Rates', color='#00809D')

# X-axis labels with group and year combined
ax.set_xticks(x)
ax.set_xticklabels([f"{g}\n({y})" for g, y in zip(groups, years)], fontsize=10)

ax.set_ylabel('Percent Increase (%)')
ax.set_title('Significant Increases in ED Visits Rates by Subgroup', fontsize=14, weight='bold')
ax.legend()

ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add percent labels on top of bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

add_labels(bars1)
add_labels(bars2)

plt.tight_layout()
plt.savefig("EDsubgroupALLRATES.png", dpi=300, bbox_inches='tight')
plt.show()

#-----------------------------------------------------------------------------------

"""
Visual 4A:breakdown for cases vs subgroup - Detailed View – Increasing Condition
Purpose: Break down significant increases in visit counts for specific conditions across subgroups.
How:
For each subgroup, plot % change over year ranges.
Bars are grouped and color-coded by subgroup.
Titles and axis labels show condition and scale.
"""


# Filtering data for the specific condition
condition_name = "9. Certain infectious and parasitic diseases"
condition_data = df[df['Condition'] == condition_name].copy()

# Getting unique year ranges and subgroups for this specific condition
year_ranges = condition_data['Year Range'].unique()
subgroups = condition_data['Subgroup'].unique()

print(f"Found {len(year_ranges)} year ranges: {year_ranges}")
print(f"Found {len(subgroups)} subgroups: {subgroups}")

# Setting up the plot
plt.figure(figsize=(15, 8))

# Creating a color palette for subgroups
colors = plt.cm.Set1(np.linspace(0, 1, len(subgroups)))
color_map = dict(zip(subgroups, colors))

# Calculate bar positions
n_years = len(year_ranges)
n_subgroups = len(subgroups)
bar_width = 0.8 / n_subgroups  # Total width of 0.8 divided by number of subgroups
x_positions = np.arange(n_years)

# Create a dictionary to track how many bars each year actually has
bars_per_year = {}
for year_range in year_ranges:
    year_data = condition_data[condition_data['Year Range'] == year_range]
    bars_per_year[year_range] = len(year_data['Subgroup'].unique())

# Plot bars for each subgroup
for i, subgroup in enumerate(subgroups):
    subgroup_data = condition_data[condition_data['Subgroup'] == subgroup]

    # Get % Change values for each year range for this subgroup
    values = []
    x_positions_adjusted = []

    for j, year_range in enumerate(year_ranges):
        year_data = subgroup_data[subgroup_data['Year Range'] == year_range]
        if len(year_data) > 0:
            # Convert % Change to numeric (remove % sign if present)
            pct_change = year_data['% Change'].iloc[0]
            if isinstance(pct_change, str) and '%' in pct_change:
                pct_change = float(pct_change.replace('%', ''))
            values.append(float(pct_change))

            # Adjust positioning based on how many bars this year actually has
            actual_bars_this_year = bars_per_year[year_range]
            if actual_bars_this_year == 1:
                # Center single bars
                x_pos = x_positions[j]
            else:
                # Use normal grouped positioning for multiple bars
                subgroups_this_year = condition_data[condition_data['Year Range'] == year_range]['Subgroup'].unique()
                subgroup_index = list(subgroups_this_year).index(subgroup)
                x_pos = x_positions[j] + (subgroup_index - actual_bars_this_year/2 + 0.5) * bar_width

            x_positions_adjusted.append(x_pos)

    # Plot bars
    if values:  # Only plot if there are values
        plt.bar(x_positions_adjusted, values, bar_width, label=subgroup, color=color_map[subgroup], alpha=0.8)

# Customize the plot
plt.xlabel('Year Range')
plt.ylabel('% Change')
# plt.title(condition_name, fontsize=14, pad=20)  # Commented out for custom title

# Ensure title has enough space
plt.subplots_adjust(top=0.75)

# Set x-axis labels
plt.xticks(x_positions, year_ranges, rotation=45, ha='right')

# Add horizontal line at y=0 for reference
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.7)

# Add legend
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
#title:
plt.suptitle("Statistically Significant Changes in ED Visit Counts by Demographics:",
             fontsize=15, fontweight='bold', y=0.99)
plt.title("Primary Diagnosis:Certain infectious and parasitic diseases",
          fontsize=14, style='italic', y=0.98, pad=20)

# Add grid for better readability
plt.grid(True, alpha=0.3, axis='y')

# Adjust layout to prevent legend cutoff
plt.tight_layout()

# Set y-axis range from 0 to 200%
plt.ylim(0, 120)

# Show the plot
plt.show()

# Print summary statistics
print(f"\nSummary for {condition_name}:")
print(f"Number of data points: {len(condition_data)}")
print(f"Year ranges covered: {', '.join(year_ranges)}")
#--------------------------------------------------------------------------------------------
"""
Visual 4B: Detailed View – Fluctuating Condition
Purpose: Show both increases and decreases (signed change) for subgroups within fluctuating conditions.
How:
Applies a sign (+/-) to % change based on "Direction" column.
Bars are sorted within each year range by magnitude.
Plots vertically with color-coded subgroups and signed bars.
"""

condition_name = "2. Injury and poisoning"  # change as needed
csv_file = 'df_detailed_primary_countn_T02.csv'

# Load data > subtables were saved in a seperate sheet
df = pd.read_csv(csv_file)

# Filter by condition
cond_df = df[df['Condition'] == condition_name].copy()

#  Prepare data: apply sign to % Change based on Direction
def signed_pct_change(row):
    val = row['% Change']
    if isinstance(val, str) and '%' in val:
        val = float(val.replace('%', ''))
    else:
        val = float(val)
    if row['Direction'].strip().lower() == 'decrease':
        return -val
    else:
        return val

cond_df['Signed Change'] = cond_df.apply(signed_pct_change, axis=1)

#  Sort data within each Year Range by abs(Signed Change), descending
sorted_groups = {}
year_ranges = sorted(cond_df['Year Range'].unique())

max_bars_per_block = 0
for yr in year_ranges:
    block_df = cond_df[cond_df['Year Range'] == yr].copy()
    block_df = block_df.reindex(block_df['Signed Change'].abs().sort_values(ascending=False).index)
    sorted_groups[yr] = block_df
    max_bars_per_block = max(max_bars_per_block, len(block_df))

# assign color per Subgroup
all_subgroups = cond_df['Subgroup'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(all_subgroups)))
color_map = dict(zip(all_subgroups, colors))

# Plot setup
plt.figure(figsize=(14, 7))

total_block_width = 0.8  # total width for bars in each year block
bar_width = total_block_width / max_bars_per_block

x_ticks = []
x_tick_labels = []
bars = []
legend_labels = []

for i, yr in enumerate(year_ranges):
    block_df = sorted_groups[yr]
    n_bars = len(block_df)
    if n_bars == 0:
        continue

    # Calculate left margin to center bars when fewer than max
    empty_space = max_bars_per_block - n_bars
    left_margin = (empty_space / 2) * bar_width

    # Positions of bars for this year block
    for j, (idx, row) in enumerate(block_df.iterrows()):
        x_pos = i + left_margin + j * bar_width
        bar = plt.bar(
            x_pos,
            row['Signed Change'],
            width=bar_width * 0.9,
            color=color_map[row['Subgroup']],
            alpha=0.85
        )
        bars.append(bar)

        if row['Subgroup'] not in legend_labels:
            legend_labels.append(row['Subgroup'])

    # Calculate and store the center x-position of this year block group (for x-axis ticks)
    center_pos = i + total_block_width / 2
    x_ticks.append(center_pos)
    x_tick_labels.append(yr)

# Customize plot

plt.axhline(0, color='gray', linewidth=1, linestyle='--', alpha=0.7)
plt.ylabel('% Change (signed by direction)')
plt.suptitle("Statistically Significant Changes in ED Visit Counts by Demographics:",
             fontsize=15, fontweight='bold', y=0.99)
plt.title("  Primary Diagnosis: Injury and poisoning",
          fontsize=14, style='italic', y=0.98, pad=20)
plt.xticks(ticks=x_ticks, labels=x_tick_labels, rotation=45, ha='right')

handles = [plt.Rectangle((0,0),1,1,color=color_map[sub]) for sub in legend_labels]
plt.legend(handles, legend_labels, bbox_to_anchor=(1.05, 1), loc='upper left', title='Subgroup')
plt.grid(axis='y', alpha=0.3)
plt.savefig("demographic_injury.png", dpi=300, bbox_inches='tight')
plt.show()
#----------------------------------------------------------------------------------------
"""
Visual 5 : Overall ED Visits (Total)

Purpose: Prepare a basic time series plot of total ED visit counts (no significance testing).
"""

# Filter the DataFrame for relevant data
df1= df_all_primary_count
filtered_df = df1 [
    (df1['Subgroup'] == 'All visits') &
    (df1['Estimate Type'] == 'Visit count')
]

# Group by Year and sum the estimates in case of duplicates
plot_data = filtered_df.groupby('Year')['Estimate'].sum().reset_index()

# Sort by Year (just in case)
plot_data = plot_data.sort_values('Year')

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(plot_data['Year'], plot_data['Estimate'], marker='o', linestyle='-', color='black')

# Title and labels
plt.title('Overall ED Visit Trend(All Primary Diagnosis Counts)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Visit Count', fontsize=12)

# Optional: Add grid
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.savefig("TPC.png", dpi=300, bbox_inches='tight')
plt.show()

